In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# `Logit` on Orders - A warm-up challenge (~1h)

## Select features

🎯 Let's figure out the impact of `wait_time` and `delay_vs_expected` on very `good/bad reviews`

👉 Using our `orders` training_set, we will run two `multivariate logistic regressions`:
- `logit_one` to predict `dim_is_one_star` 
- `logit_five` to predict `dim_is_five_star`.

 

In [25]:
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

👉 Import your dataset:

In [26]:
from olist.order import Order
orders = Order().get_training_data(with_distance_seller_customer=True)

/Users/taniagretton/code/TaniaGretton/data-context-and-setup/olist/order.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'],format='%Y-%m-%d %H:%M:%S')
/Users/taniagretton/code/TaniaGretton/data-context-and-setup/olist/order.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders['order_approved_at'] = pd.to_datetime(orders['order_approved_at'],format='%Y-%m-%d %H:%M:%S')
/Users/taniagretton/code/TaniaGretton/data-c

👉 Select in a list which features you want to use:

⚠️ Make sure you are not creating data leakage (i.e. selecting features that are derived from the target)

💡 To figure out the impact of `wait_time` and `delay_vs_expected` we need to control for the impact of other features, include in your list all features that may be relevant

In [27]:
# YOUR CODE HERE
features = ['dim_is_one_star', 'dim_is_five_star','wait_time','delay_vs_expected','distance_seller_customer','review_score']
features

['dim_is_one_star',
 'dim_is_five_star',
 'wait_time',
 'delay_vs_expected',
 'distance_seller_customer',
 'review_score']

In [28]:
orders.head()

,order_id,wait_time,expected_wait_time,delay_vs_expected,order_status,dim_is_five_star,dim_is_one_star,review_score,number_of_products,number_of_sellers,price,freight_value,distance_seller_customer
0,e481f51cbdc54678b7cc49136f2d6af7,8.436574,15.544063,0.0,delivered,0,0,4,1,1,29.99,8.72,18.063837
1,53cdb2fc8bc7dce0b6741e2150273451,13.782037,19.137766,0.0,delivered,0,0,4,1,1,118.70,22.76,856.292580
2,47770eb9100c2d0c44946d9cf07ec65d,9.394213,26.639711,0.0,delivered,1,0,5,1,1,159.90,19.22,514.130333
3,949d5b44dbf5de918fe9c16f97b45f8a,13.208750,26.188819,0.0,delivered,1,0,5,1,1,45.00,27.20,1822.800366
4,ad21c59c0840e6cb83a9ceb5573f8159,2.873877,12.112049,0.0,delivered,1,0,5,1,1,19.90,8.72,30.174037


🕵🏻 Check the `multi-colinearity` of your features, using the `VIF index`.

* It shouldn't be too high (< 10 preferably) to ensure that we can trust the partial regression coefficents and their associated `p-values` 
* Do not forget to standardize your data ! 
    * A `VIF Analysis` is made by regressing a feature vs. the other features...
    * So you want to `remove the effect of scale` so that your features have an equal importance before running any linear regression!
    
    
📚 <a href="https://www.statisticshowto.com/variance-inflation-factor/">Statistics How To - Variance Inflation Factor</a>

📚  <a href="https://online.stat.psu.edu/stat462/node/180/">PennState - Detecting Multicollinearity Using Variance Inflation Factors</a>

⚖️ Standardizing:

In [29]:
# YOUR CODE HERE
orders_scaled = orders.copy()
orders_scaled = orders[['dim_is_one_star','dim_is_five_star','wait_time','delay_vs_expected','distance_seller_customer','review_score']]
#orders_scaled
for feature in orders_scaled.columns:
    mu = orders[feature].mean()
    sigma = orders[feature].std()
    orders_scaled[feature] = orders_scaled[feature].apply(lambda x: (x-mu)/sigma)
    
orders_scaled

/var/folders/vp/ynvw_5l50czfjyr3gfjrkl6m0000gn/T/ipykernel_44610/4272022337.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_scaled[feature] = orders_scaled[feature].apply(lambda x: (x-mu)/sigma)


,dim_is_one_star,dim_is_five_star,wait_time,delay_vs_expected,distance_seller_customer,review_score
0,-0.328964,-1.204841,-0.431192,-0.161781,-0.979475,-0.121008
1,-0.328964,-1.204841,0.134174,-0.161781,0.429743,-0.121008
2,-0.328964,0.829977,-0.329907,-0.161781,-0.145495,0.657133
3,-0.328964,0.829977,0.073540,-0.161781,2.054621,0.657133
4,-0.328964,0.829977,-1.019535,-0.161781,-0.959115,0.657133
...,...,...,...,...,...,...
95875,-0.328964,0.829977,-0.454309,-0.161781,-0.893033,0.657133
95876,-0.328964,-1.204841,1.023841,-0.161781,-0.212797,-0.121008
95877,-0.328964,0.829977,1.305780,-0.161781,0.617630,0.657133
95878,-0.328964,-1.204841,0.483664,-0.161781,-0.387558,-1.677291


👉 Run your VIF Analysis to analyze the potential multicolinearities:

In [30]:
# YOUR CODE HERE
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
# compute VIF factor for feature index 0
vif(orders_scaled.values, 0)

5.311587431162536

In [31]:
df = pd.DataFrame()

df["features"] = orders_scaled.columns

df["vif_index"] = [vif(orders_scaled.values, i) for i in range(orders_scaled.shape[1])]

round(df.sort_values(by="vif_index", ascending = False),2)

,features,vif_index
5,review_score,12.05
0,dim_is_one_star,5.31
1,dim_is_five_star,4.93
2,wait_time,2.73
3,delay_vs_expected,2.23
4,distance_seller_customer,1.33


## Logistic Regressions

👉 Fit two `Logistic Regression` models:
- `logit_one` to predict `dim_is_one_star` 
- `logit_five` to predict `dim_is_five_star`.

`Logit 1️⃣`

In [41]:
# YOUR CODE HERE
logit1 = smf.logit(formula='dim_is_one_star ~ wait_time + delay_vs_expected + distance_seller_customer', data=orders).fit()
logit1.params

Optimization terminated successfully.
         Current function value: 0.282203
         Iterations 7


Intercept                  -3.158450
wait_time                   0.070792
delay_vs_expected           0.056105
distance_seller_customer   -0.000296
dtype: float64

`Logit 5️⃣`

In [42]:
# YOUR CODE HERE
logit5 = smf.logit(formula='dim_is_five_star ~ wait_time + delay_vs_expected + distance_seller_customer', data=orders).fit()
logit1.params

Optimization terminated successfully.
         Current function value: 0.642018
         Iterations 7


Intercept                  -3.158450
wait_time                   0.070792
delay_vs_expected           0.056105
distance_seller_customer   -0.000296
dtype: float64

💡 It's time to analyse the results of these two logistic regressions:

- Interpret the partial coefficients in your own words.
- Check their statistical significances with `p-values`
- Do you notice any differences between `logit_one` and `logit_five` in terms of coefficient importances?

In [39]:
# Among the following sentences, store the ones that are true in the list below

a = "delay_vs_expected influences five_star ratings even more than one_star ratings"
b = "wait_time influences five_star ratings even more more than one_star"

your_answer = [a]

🧪 __Test your code__

In [40]:
from nbresult import ChallengeResult

result = ChallengeResult('logit',
    answers = your_answer
)
result.write()
print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /Users/taniagretton/.pyenv/versions/3.10.6/envs/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /Users/taniagretton/code/TaniaGretton/data-logit/tests
plugins: asyncio-0.19.0, typeguard-2.13.3, anyio-3.6.2
asyncio: mode=strict
collecting ... collected 1 item

test_logit.py::TestLogit::test_question PASSED                           [100%]

============================== 1 passed in 0.01s ===============================


💯 You can commit your code:

git add tests/logit.pickle

git commit -m 'Completed logit step'

git push origin master



<details>
    <summary>- <i>Explanations and advanced concepts </i> -</summary>


> _All other thing being equal, the `delay factor` tends to increase the chances of getting stripped of the 5-star even more so than it affect the chances of 1-star reviews. Probably because 1-stars are really targeting bad products themselves, not bad deliveries_
    
❗️ However, to be totally rigorous, we have to be **more careful when comparing coefficients from two different models**, because **they might not be based on similar populations**!
    We have 2 sub-populations here: (people who gave 1-stars; and people who gave 5-stars) and they may exhibit intrinsically different behavior patterns. It may well be that "happy-people" (who tends to give 5-stars easily) are less sensitive as "grumpy-people" (who shoot 1-stars like Lucky-Luke), when it comes to "delay", or "price"...

</details>


## Logistic vs. Linear ?

👉 Compare the coefficients obtained from:
- A `Logistic Regression` to explain `dim_is_five_star`
- A `Linear Regression` to explain `review_score` 

Make sure to use the same set of features for both regressions.  

⚠️ Check that both sets of coefficients  tell  "the same story".

> YOUR ANSWER HERE

In [ ]:
# YOUR CODE HERE

🏁 Congratulations! 

💾 Don't forget to commit and push your `logit.ipynb` notebook !